In [4]:
%reset -f
%pip install -r requirement2.txt



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import pandas as pd
import numpy as np 
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from datetime import datetime, timedelta
import logging


In [6]:
load_dotenv()

DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

SOURCE_TABLE_A = "DS_BKC_Infineon_TMP-NVD-01_BY_LOT_Daily_6AM"
SOURCE_TABLE_B = "DS_BKC_MSP_Yield_Lot"
TARGET_TABLE = "DS_BKC_PACK_MSP"

In [7]:
from datetime import datetime, timedelta, date
import logging

# Configure logging if you want to see the info message
# logging.basicConfig(level=logging.INFO)

def get_relevant_week_range_sat_fri():
    """
    Calculates the start (Saturday) and end date (Friday) of the week
    containing the specified latest date from the data.

    Args:
        latest_data_date_str (str): The latest date available in your data
                                    in 'YYYY-MM-DD' format.
    """

    latest_data_date_str = "2025-04-03"
    try:
        # Use the latest date from your data as the reference point
        # Example: '2025-04-03' based on your description
        reference_day = datetime.strptime(latest_data_date_str, '%Y-%m-%d').date()
    except ValueError:
        logging.error("Invalid date format provided. Please use YYYY-MM-DD.")
        return None, None # Or raise an exception

    # Calculate days to subtract to get to the *previous* Saturday
    # reference_day.weekday(): Mon=0, Tue=1, Wed=2, Thu=3, Fri=4, Sat=5, Sun=6
    # We want Saturday (5) to be day 0 of our target week.
    # Offset: Sat=0, Sun=1, Mon=2, Tue=3, Wed=4, Thu=5, Fri=6
    days_since_last_saturday = (reference_day.weekday() + 2) % 7
    start_of_week = reference_day - timedelta(days=days_since_last_saturday) # Saturday
    end_of_week = start_of_week + timedelta(days=6) # Friday

    logging.info(f"Relevant Sat-Fri week range based on data ending {reference_day}: {start_of_week} to {end_of_week}")
    return start_of_week, end_of_week

# --- How to use it ---
# Your data ends on April 3, 2025
latest_date_in_data = "2025-04-03"
start_date, end_date = get_relevant_week_range_sat_fri()

if start_date and end_date:
    print(f"The week range containing the latest data is: {start_date} to {end_date}")
    # Now use start_date and end_date to query your data

    # For April 3, 2025 (a Thursday):
    # days_since_last_saturday = (3 + 2) % 7 = 5
    # start_of_week = 2025-04-03 - 5 days = 2025-03-29 (Saturday)
    # end_of_week = 2025-03-29 + 6 days = 2025-04-04 (Friday)
    # Expected output: The week range containing the latest data is: 2025-03-29 to 2025-04-04

The week range containing the latest data is: 2025-03-29 to 2025-04-04


In [8]:
def get_current_week_range_sat_fri():
    """
    Calculates the start (Saturday) and end date (Friday) of the week
    containing the current day.
    """
    today = datetime.now().date()
    # Calculate days to subtract to get to the *previous* Saturday
    # today.weekday(): Mon=0, Tue=1, Wed=2, Thu=3, Fri=4, Sat=5, Sun=6
    # We want Saturday (5) to be day 0 of our target week.
    # Offset: Sat=0, Sun=1, Mon=2, Tue=3, Wed=4, Thu=5, Fri=6
    days_since_last_saturday = (today.weekday() + 2) % 7
    start_of_week = today - timedelta(days=days_since_last_saturday) # Saturday
    end_of_week = start_of_week + timedelta(days=6) # Friday
    logging.info(f"Current Sat-Fri week range: {start_of_week} to {end_of_week}")
    return start_of_week, end_of_week

In [9]:
try:
    # Construct the database URL
    db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    
    # Create the engine
    engine = create_engine(db_url)
    
    # Connect to the database
    with engine.connect() as connection:
        logging.info("Successfully connected to the PostgreSQL database.")

        print('Successfully connected to the PostgreSQL database.')
        # Execute a simple query to test the connection
        connection.execute(text("SELECT 1"))
except Exception as e:
    logging.error(f"Database connection failed: {e}")

Successfully connected to the PostgreSQL database.


In [10]:
start_of_week, end_of_week = get_relevant_week_range_sat_fri()

cols_a = ['LOTID', 'PKGLD', 'PACKINGTYPE', 'QTY', 'TYPE', 'PRODUCTNAME', 'DATESTAMP']
cols_a_str = ", ".join([f'"{col}"' for col in cols_a])
query_a = f"""
        SELECT {cols_a_str}
        FROM "{SOURCE_TABLE_A}"
        WHERE "DATESTAMP"::date >= '{start_of_week}'
          AND "DATESTAMP"::date <= '{end_of_week}'                                                  
          AND "TYPE" = '16 P/O' -- Filter TYPE in SQL for efficiency
        """
      
df_a = pd.read_sql(query_a, engine, parse_dates=['DATESTAMP'])
df_a

,LOTID,PKGLD,PACKINGTYPE,QTY,TYPE,PRODUCTNAME,DATESTAMP
0,1S524137EM1,TS 056,0010 TRAY,2730,16 P/O,98741B2 CAPF-82538 06M,2025-03-29 02:02:48
1,1S522371EM1,TS 048,00 TRAY PKGED,11520,16 P/O,98K38B2BCP29A16J7FI020 06C,2025-03-29 03:06:21
2,1S524528EN9,TS 056,0002 TRAY,182,16 P/O,98871A1 CJPF-47749 06E,2025-03-29 00:09:55
3,1S525116EM1,VBM063,03 13 IN. TAPE&REEL,20700,16 P/O,98799A1EGHPF-41205 06D,2025-03-29 00:52:51
4,1S524628EM1,FAB024,03 13 IN. TAPE&REEL,12500,16 P/O,98UZ0A3NB8PF-85170 06M,2025-03-29 01:30:30
...,...,...,...,...,...,...,...
1694,1S526157EM1,SOC008,0010 TRAY,25200,16 P/O,98RZ9A OCPF-49935 06C,2025-03-29 11:35:05
1695,1S522699EM1,FAB024,03 13 IN. TAPE&REEL,42500,16 P/O,98UZ0A2NVEPF-84481 06M,2025-03-29 08:34:33
1696,1S524286EM1,FAB024,03 13 IN. TAPE&REEL,42500,16 P/O,98UZ0A2NVEPF-85169 06M,2025-03-29 07:51:09
1697,1S526080EM1,SOC008,03 13 IN. TAPE&REEL,25200,16 P/O,98RZ9A OCPF-51212 06M,2025-03-29 17:17:35


In [14]:
agg_cols_a = [col for col in df_a.columns if col != 'LOTID']
agg_dict_a = {col: 'max' for col in agg_cols_a}
df_a_agg = df_a.groupby('LOTID', as_index=False).agg(agg_dict_a)
df_a_agg = df_a_agg[['LOTID'] + agg_cols_a] 

In [13]:
cols_b = ['LOT', 'MOVEOUT', 'SPECNAME', 'MACHINE']
cols_b_str = ", ".join([f'"{col}"' for col in cols_b])
specname_filter = ['Pack', 'Pack1', 'Pack1_TR', 'PACK_TR']
query_b = f"""
        SELECT {cols_b_str}
        FROM "{SOURCE_TABLE_B}"
        WHERE "MOVEOUT"::date >= '{start_of_week}'
          AND "MOVEOUT"::date <= '{end_of_week}'
          AND "SPECNAME" IN ({", ".join([f"'{name}'" for name in specname_filter])})
        """
df_b = pd.read_sql(query_b, engine, parse_dates=['MOVEOUT'])
df_b

,LOT,MOVEOUT,SPECNAME,MACHINE
0,1S526518,2025-03-29 02:11:28,Pack,PACK-TRAY03
1,1S50503NEM3,2025-03-31 20:13:16,Pack,PACK-TRAY03
2,1S526114EM1,2025-04-02 12:41:23,Pack,PACK-TRAY03
3,1S526007EM1,2025-03-30 02:16:06,Pack,PACK-TRAY03
4,1S524658EM1,2025-04-01 09:05:57,Pack,PACK-TRAY02
...,...,...,...,...
241,1S527468,2025-04-02 15:58:29,Pack,PACK-TRAY02
242,1S527467,2025-04-02 15:39:22,Pack,PACK-TRAY02
243,1S527465,2025-04-02 15:50:31,Pack,PACK-TRAY03
244,1S527466,2025-04-02 16:10:57,Pack,PACK-TRAY03


In [11]:
import pandas as pd

In [12]:
        # --- 2. Extract and Transform DS_BKC_MD_Product ---
        
md_product_query = """
        SELECT "PRODUCT", "WORKFLOWNAME", "ISROR", "IFXFUNCTIONALPACK", 
               "DEFAULTSTARTOWNER", "ISAVAILABLE", "IFXMANUFACTURINGLEVEL"
        FROM "DS_BKC_MD_Product"
        WHERE "ISAVAILABLE" = 'True'
          AND "IFXMANUFACTURINGLEVEL" = 'TEST'
          AND "ISROR" = 'True'
          AND "DEFAULTSTARTOWNER" = 'PROD'
        """
df_md = pd.read_sql(md_product_query, engine)
df_md

,PRODUCT,WORKFLOWNAME,ISROR,IFXFUNCTIONALPACK,DEFAULTSTARTOWNER,ISAVAILABLE,IFXMANUFACTURINGLEVEL
0,11035A HHPF-46981 06E,T_1T_NTR_BINSP_TBM_1QA,True,0001 TRAY,PROD,True,TEST
1,11035A HHPF-49433 06M,T_2T_NTR_BINSP_TBM_1QA,True,0005 TRAY,PROD,True,TEST
2,11036A HHPF-49412 06M,T_2T_NTR_BINSP_TBM_1QA,True,0005 TRAY,PROD,True,TEST
3,11036A HHPF-49413 06M,T_2T_TR_BINSP_TBM_1QA_KLA,True,03 13 IN. TAPE&REEL,PROD,True,TEST
4,11040A HHPF-46975 06C,T_1T_NTR_BINSP_TBM_1QA,True,0002 TRAY,PROD,True,TEST
...,...,...,...,...,...,...,...
5076,U289A2TLGHPF-84241 06M,T_3T_TR_BINSP_TBM_1QA_KLA,True,03 13 IN. TAPE&REEL,PROD,True,TEST
5077,U462A1UPCAPF-17231 06M,T_2T_NAND_NTR_LINSP_TBM_1QA,True,00 TRAY PKGED,PROD,True,TEST
5078,U462A1UPCAPF-17232 06M,T_2T_NAND_TR_LINSP_TBM_1QA,True,03 13 IN. TAPE&REEL,PROD,True,TEST
5079,U462A1UPCAPF-39716 06M,T_2T_NAND_NTR_LINSP_TBM_1QA,True,00 TRAY PKGED,PROD,True,TEST


In [15]:
df_merged = pd.merge(
    df_a_agg,
    df_b, 
    left_on='LOTID',
    right_on='LOT',
    how='inner'
        )
df_merged

,LOTID,PKGLD,PACKINGTYPE,QTY,TYPE,PRODUCTNAME,DATESTAMP,LOT,MOVEOUT,SPECNAME,MACHINE
0,1S5040QVEM1,TS 056,0010 TRAY,170,16 P/O,98290B2LCJPF-56728 06M,2025-03-29 08:10:00,1S5040QVEM1,2025-03-29 08:05:54,Pack,PACK-TRAY02
1,1S504534EQ6,TS 056,0010 TRAY,399,16 P/O,98223B2LCAPF-85985 06C,2025-03-29 08:20:33,1S504534EQ6,2025-03-29 08:15:49,Pack,PACK-TRAY02
2,1S504538EQ9,TS 056,0010 TRAY,735,16 P/O,98223BL CJPF-57215 06M,2025-03-29 08:44:22,1S504538EQ9,2025-03-29 08:40:07,Pack,PACK-TRAY02
3,1S50503NEM3,VAA024,0010 TRAY,274,16 P/O,98UZ0A2MLABPF-47707 06M,2025-03-31 20:58:43,1S50503NEM3,2025-03-31 20:13:16,Pack,PACK-TRAY03
4,1S505706EM2,TS 056,0010 TRAY,210,16 P/O,98223B2LCJPF-57215 06M,2025-03-29 08:02:33,1S505706EM2,2025-03-29 07:55:07,Pack,PACK-TRAY02
...,...,...,...,...,...,...,...,...,...,...,...
218,1S527118EM1,SO3016,0010 TRAY,16800,16 P/O,98222AU OCPF-50936 06M,2025-04-02 22:25:37,1S527118EM1,2025-04-02 22:09:20,Pack,PACK-TRAY02
219,1S527119EM1,SO3016,0010 TRAY,16800,16 P/O,98222AU OCPF-50936 06M,2025-04-03 21:17:16,1S527119EM1,2025-04-03 20:58:05,Pack,PACK-TRAY02
220,1S527167EM1,SO3016,0010 TRAY,26400,16 P/O,98222AU OCPF-50936 06M,2025-04-04 02:00:56,1S527167EM1,2025-04-04 01:02:08,Pack,PACK-TRAY02
221,1S527173EM1,SO3016,0010 TRAY,24000,16 P/O,98222AU OCPF-50936 06M,2025-04-03 14:28:24,1S527173EM1,2025-04-03 14:06:33,Pack,PACK-TRAY02


In [17]:

df_merged = pd.merge(
df_merged,
            df_md,
            how='left',
            left_on=['PRODUCTNAME', 'PACKINGTYPE'],
            right_on=['PRODUCT', 'IFXFUNCTIONALPACK']
        )
df_merged
        


,LOTID,PKGLD,PACKINGTYPE,QTY,TYPE,PRODUCTNAME,DATESTAMP,LOT,MOVEOUT,SPECNAME,...,DEFAULTSTARTOWNER_x,ISAVAILABLE_x,IFXMANUFACTURINGLEVEL_x,PRODUCT_y,WORKFLOWNAME_y,ISROR_y,IFXFUNCTIONALPACK_y,DEFAULTSTARTOWNER_y,ISAVAILABLE_y,IFXMANUFACTURINGLEVEL_y
0,1S5040QVEM1,TS 056,0010 TRAY,170,16 P/O,98290B2LCJPF-56728 06M,2025-03-29 08:10:00,1S5040QVEM1,2025-03-29 08:05:54,Pack,...,PROD,True,TEST,98290B2LCJPF-56728 06M,T_2T_NTR_LINSP_TBM_1QA,True,0010 TRAY,PROD,True,TEST
1,1S504534EQ6,TS 056,0010 TRAY,399,16 P/O,98223B2LCAPF-85985 06C,2025-03-29 08:20:33,1S504534EQ6,2025-03-29 08:15:49,Pack,...,PROD,True,TEST,98223B2LCAPF-85985 06C,T_1T_NTR_LINSP_TBM_1QA,True,0010 TRAY,PROD,True,TEST
2,1S504538EQ9,TS 056,0010 TRAY,735,16 P/O,98223BL CJPF-57215 06M,2025-03-29 08:44:22,1S504538EQ9,2025-03-29 08:40:07,Pack,...,PROD,True,TEST,98223BL CJPF-57215 06M,T_2T_NTR_LINSP_TBM_1QA,True,0010 TRAY,PROD,True,TEST
3,1S50503NEM3,VAA024,0010 TRAY,274,16 P/O,98UZ0A2MLABPF-47707 06M,2025-03-31 20:58:43,1S50503NEM3,2025-03-31 20:13:16,Pack,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1S505706EM2,TS 056,0010 TRAY,210,16 P/O,98223B2LCJPF-57215 06M,2025-03-29 08:02:33,1S505706EM2,2025-03-29 07:55:07,Pack,...,PROD,True,TEST,98223B2LCJPF-57215 06M,T_2T_NTR_LINSP_TBM_1QA,True,0010 TRAY,PROD,True,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,1S527118EM1,SO3016,0010 TRAY,16800,16 P/O,98222AU OCPF-50936 06M,2025-04-02 22:25:37,1S527118EM1,2025-04-02 22:09:20,Pack,...,PROD,True,TEST,98222AU OCPF-50936 06M,"SUB_ASSYMARK2, T_2T_NTR_LINSP_ETM_MBT_1QA",True,0010 TRAY,PROD,True,TEST
219,1S527119EM1,SO3016,0010 TRAY,16800,16 P/O,98222AU OCPF-50936 06M,2025-04-03 21:17:16,1S527119EM1,2025-04-03 20:58:05,Pack,...,PROD,True,TEST,98222AU OCPF-50936 06M,"SUB_ASSYMARK2, T_2T_NTR_LINSP_ETM_MBT_1QA",True,0010 TRAY,PROD,True,TEST
220,1S527167EM1,SO3016,0010 TRAY,26400,16 P/O,98222AU OCPF-50936 06M,2025-04-04 02:00:56,1S527167EM1,2025-04-04 01:02:08,Pack,...,PROD,True,TEST,98222AU OCPF-50936 06M,"SUB_ASSYMARK2, T_2T_NTR_LINSP_ETM_MBT_1QA",True,0010 TRAY,PROD,True,TEST
221,1S527173EM1,SO3016,0010 TRAY,24000,16 P/O,98222AU OCPF-50936 06M,2025-04-03 14:28:24,1S527173EM1,2025-04-03 14:06:33,Pack,...,PROD,True,TEST,98222AU OCPF-50936 06M,"SUB_ASSYMARK2, T_2T_NTR_LINSP_ETM_MBT_1QA",True,0010 TRAY,PROD,True,TEST


In [ ]:
        # --- 4. Read SHIFT_TOMORROW_7Days.csv and join ---
shift_csv_path = "SHIFT_TOMORROW_7Days.csv"
    if os.path.exists(shift_csv_path):
            logging.info(f"Reading shift info from {shift_csv_path}...")
            df_shift = pd.read_csv(shift_csv_path)

            # Ensure column casing/format match
        if 'LOT' not in df_shift.columns:
                logging.warning("Column 'LOT' not found in SHIFT_TOMORROW_7Days.csv.")
        else:
                df_merged = pd.merge(
                    df_merged,
                    df_shift,
                    how='left',
                    left_on='LOTID',
                    right_on='LOT'
                )
                logging.info("Successfully joined SHIFT_TOMORROW shift data.")
        else:
            logging.warning(f"{shift_csv_path} not found. Skipping shift data join.")
